## Prototype Solar Collector

In [ ]:
from collections import defaultdict
import casadi as cas
import numpy as np
import matplotlib.pyplot as plt
import scipy

In [ ]:
n_segments = 100

In [ ]:
RADIUS = 0.07  # metres
LENGTH = 96.0  # metres

def calculate_time_interval(flow_rate, n_segments, radius=RADIUS, length=LENGTH, pi=cas.pi):
    return pi * radius**2 * length / (n_segments * flow_rate)

# Test calculation
flow_rate = 0.9  # m^3/s
n_segments = 10
dt = calculate_time_interval(flow_rate, n_segments)
dt

## Symbolic Manipulation

In [ ]:
import sympy

I_c, epsilon, h, T_p, T_f, h_amb, A, T_a = \
    sympy.symbols('I_c, epsilon, h, T_p, T_f, h_amb, A, T_a')

eqn1 = sympy.Eq(I_c * epsilon * A / 2, h * A * (T_p - T_f) + h_amb * A * (T_p - T_a))
eqn1

In [ ]:
T_p_expr = sympy.solve(eqn1, T_p)[0]
sympy.Eq(T_p, T_p_expr)

## Thermal Collector Model

In [ ]:
params = {
    'alpha': 0.0,
    'collector_efficiency': 0.7,
    'pipe_radius': RADIUS,
    'oil_density': 800,  # kg/m^3
    'oil_heat_capacity': 2000,  # J/(kg*K)
    'pipe_int_ht_coeff': 10,  # W/(m^2*K)
    'pipe_ext_ht_coeff': 15,  # W/(m^2*K)
}

In [ ]:
def calculate_pipe_temp(solar_irradiance, oil_temp, amb_temp, params):
    pipe_int_ht_coeff = params['pipe_int_ht_coeff']
    pipe_ext_ht_coeff = params['pipe_ext_ht_coeff']
    collector_efficiency = params['collector_efficiency']
    return (
        solar_irradiance * collector_efficiency / 2 
        + pipe_int_ht_coeff * (oil_temp + 273.15)
        + pipe_ext_ht_coeff * (amb_temp + 273.15)
    ) / (pipe_int_ht_coeff + pipe_ext_ht_coeff) - 273.15

# Test calculation
solar_irradiance = 1000  # W/m^2
oil_temp = 400  # deg C
amb_temp = 25  # deg C
pipe_temp = calculate_pipe_temp(solar_irradiance, oil_temp, amb_temp, params)
pipe_temp

In [ ]:
def calculate_oil_temp_ip1_kp1(
    solar_irradiance, 
    oil_temp_im1, 
    oil_temp_i, 
    oil_temp_ip1, 
    pipe_temp, 
    dt, 
    params
):
    alpha = params['alpha']
    collector_efficiency = params['collector_efficiency']
    pipe_radius = params['pipe_radius']
    oil_density = params['oil_density']
    oil_heat_capacity = params['oil_heat_capacity']
    pipe_ext_ht_coeff = params['pipe_ext_ht_coeff']
    b = 2 * pipe_ext_ht_coeff / (pipe_radius * oil_density * oil_heat_capacity)
    a = solar_irradiance * collector_efficiency / (pipe_radius * oil_density * oil_heat_capacity)
    return (
        oil_temp_i + (
            a 
            - b * (pipe_temp - oil_temp_i)
        ) * dt 
        - alpha * (oil_temp_ip1 - 2 * oil_temp_i + oil_temp_im1)
    )

# Test calculation
solar_irradiance = 1000  # W/m^2
oil_temp_im1 = 149
oil_temp_i = 150  # deg C
oil_temp_ip1 = 151
amb_temp = 25  # deg C
pipe_temp = calculate_pipe_temp(solar_irradiance, oil_temp, amb_temp, params)
flow_rate = 0.5  # m^3/s
n_segments = 10
dt = calculate_time_interval(flow_rate, n_segments)

oil_temp_ip1_kp1 = calculate_oil_temp_ip1_kp1(
    solar_irradiance, 
    oil_temp_im1, 
    oil_temp_i, 
    oil_temp_ip1, 
    pipe_temp, 
    dt, 
    params
)
oil_temp_ip1_kp1

In [ ]:
def make_calculate_oil_temps_kp1(n_segments, params):
    oil_inlet_temp = cas.SX.sym('oil_inlet_temp')
    oil_temps = cas.SX.sym('oil_temps', n_segments)
    solar_irradiance = cas.SX.sym('solar_irradiance')
    amb_temp = cas.SX.sym('amb_temp')
    flow_rate = cas.SX.sym('flow_rate')

    pipe_temps = calculate_pipe_temp(
        solar_irradiance, oil_temps, amb_temp, params
    )

    dt = calculate_time_interval(flow_rate, n_segments)

    oil_temp_im1 = oil_inlet_temp
    oil_temps_kp1 = [oil_temp_im1]
    for i in range(n_segments - 1):
        oil_temp_ip1_kp1 = calculate_oil_temp_ip1_kp1(
            solar_irradiance,
            oil_temp_im1,
            oil_temps[i],
            oil_temps[i+1],
            pipe_temps[i],
            dt,
            params
        )
        oil_temps_kp1.append(oil_temp_ip1_kp1)
        oil_temp_im1 = oil_temps[i]
    oil_temps_kp1 = cas.vcat(oil_temps_kp1)
    assert oil_temps_kp1.shape == (n_segments, 1)

    return cas.Function(
        'calculate_oil_temps_kp1', 
        [oil_inlet_temp, oil_temps, solar_irradiance, amb_temp, flow_rate], 
        [dt, oil_temps_kp1]
    )

n_segments = 100
calculate_oil_temps_kp1 = make_calculate_oil_temps_kp1(n_segments, params)

# Test calculation
solar_irradiance = 1000  # W/m^2
oil_inlet_temp = 273
oil_temps = cas.DM(np.full(n_segments, 273))  # deg C
amb_temp = 25  # deg C
flow_rate = 0.2  # m^3/s

dt, oil_temps_kp1 = calculate_oil_temps_kp1(oil_inlet_temp, oil_temps, solar_irradiance, amb_temp, flow_rate)
dt, oil_temps_kp1

In [ ]:
t_stop = 60  # seconds

solar_irradiance = 900  # W/m^2
oil_inlet_temp = 273
oil_temps = cas.DM(np.full(n_segments, 273))  # deg C
amb_temp = 25  # deg C
flow_rate = 0.2  # m^3/s

# Dictionary to store results
results = defaultdict(list)

times = [0.0]
t = 0.0
while True:
    results['time'].append(t)
    results['oil_temps'].append(np.array(oil_temps).flatten())
    dt, oil_temps = calculate_oil_temps_kp1(oil_inlet_temp, oil_temps, solar_irradiance, amb_temp, flow_rate)
    t += float(dt)
    if t > t_stop:
        break

oil_temps = np.stack(results['oil_temps'])
times = np.array(results['time'])

x = np.linspace(0, LENGTH, n_segments)

fig = plt.figure(figsize=(7, 2.5))

plot_interval = 1. # seconds
t_plot = 0.0
for i, t in enumerate(times):
    if t >= t_plot:
        plt.plot(x, oil_temps[i], color='tab:blue', alpha=0.5, label=f'{t:.1f} s')
        t_plot += plot_interval
    # if i >= 20:
    #     break

plt.grid()
plt.xlabel('Position along line (m)')
plt.ylabel('Oil Temperature (deg C)')
plt.title('Evolution of Oil Temperatures in Solar Collector Line')
plt.tight_layout()
plt.show()

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np
from collections import defaultdict


def create_solar_collector_animation(results, n_segments=10, pipe_length=10.0):
    """
    Create an animated Plotly visualization of solar collector oil temperatures
    
    Parameters:
    -----------
    results : dict
        Dictionary containing 'time' and 'oil_temps' arrays
    n_segments : int
        Number of pipe segments
    pipe_length : float
        Total pipe length in meters
    """
    
    # Extract data
    times = np.array(results['time'])
    oil_temps_array = np.array(results['oil_temps'])
    
    # Create position array (distance along pipe)
    positions = np.linspace(0, pipe_length, n_segments)
    
    print(f"Creating animation with {len(times)} time steps")
    print(f"Temperature range: {oil_temps_array.min():.1f} - {oil_temps_array.max():.1f} K")
    
    # Create the animated figure
    fig = go.Figure()
    
    # Add traces for each time step
    for i, t in enumerate(times):
        fig.add_trace(
            go.Scatter(
                x=positions,
                y=oil_temps_array[i],
                mode='lines+markers',
                name=f't = {t:.1f}s',
                line=dict(width=3, color='red'),
                marker=dict(size=8, color='darkred'),
                visible=False  # Initially hide all traces
            )
        )
    
    # Make first trace visible
    fig.data[0].visible = True
    
    # Create animation frames
    frames = []
    for i, t in enumerate(times):
        frame = go.Frame(
            data=[
                go.Scatter(
                    x=positions,
                    y=oil_temps_array[i],
                    mode='lines+markers',
                    line=dict(width=3, color='red'),
                    marker=dict(size=8, color='darkred'),
                    name=f't = {t:.1f}s'
                )
            ],
            name=str(i),
            layout=go.Layout(
                title=f"Solar Collector Oil Temperature - Time: {t:.1f}s",
                annotations=[
                    dict(
                        x=0.02, y=0.98,
                        xref="paper", yref="paper",
                        text=f"Max Temp: {(oil_temps_array[i]).max():.1f}°C",
                        showarrow=False,
                        bgcolor="white",
                        bordercolor="black",
                        borderwidth=1
                    ),
                    dict(
                        x=0.02, y=0.92,
                        xref="paper", yref="paper", 
                        text=f"Outlet Temp: {oil_temps_array[i][-1]:.1f}°C",
                        showarrow=False,
                        bgcolor="white",
                        bordercolor="black",
                        borderwidth=1
                    )
                ]
            )
        )
        frames.append(frame)
    
    fig.frames = frames
    
    # Update layout
    fig.update_layout(
        title="Solar Collector Oil Temperature Evolution",
        xaxis_title="Position along pipe [m]",
        yaxis_title="Oil Temperature [°C]",
        width=800,
        height=500,
        showlegend=False,
        updatemenus=[
            {
                "type": "buttons",
                "direction": "left",
                "buttons": [
                    {
                        "args": [None, {
                            "frame": {"duration": 200, "redraw": True},
                            "fromcurrent": True,
                            "transition": {"duration": 100}
                        }],
                        "label": "Play",
                        "method": "animate"
                    },
                    {
                        "args": [[None], {
                            "frame": {"duration": 0, "redraw": True},
                            "mode": "immediate",
                            "transition": {"duration": 0}
                        }],
                        "label": "Pause",
                        "method": "animate"
                    }
                ],
                "pad": {"r": 10, "t": 87},
                "showactive": False,
                "x": 0.011,
                "xanchor": "right",
                "y": 0,
                "yanchor": "top"
            }
        ],
        sliders=[{
            "active": 0,
            "yanchor": "top",
            "xanchor": "left",
            "currentvalue": {
                "font": {"size": 20},
                "prefix": "Time: ",
                "suffix": "s",
                "visible": True,
                "xanchor": "right"
            },
            "transition": {"duration": 100},
            "pad": {"b": 10, "t": 50},
            "len": 0.9,
            "x": 0.1,
            "y": 0,
            "steps": [
                {
                    "args": [[str(i)], {
                        "frame": {"duration": 100, "redraw": True},
                        "mode": "immediate",
                        "transition": {"duration": 100}
                    }],
                    "label": f"{t:.1f}",
                    "method": "animate"
                }
                for i, t in enumerate(times)
            ]
        }]
    )
    
    # Set consistent y-axis range
    y_min = (oil_temps_array).min() - 5
    y_max = (oil_temps_array).max() + 5
    fig.update_yaxes(range=[y_min, y_max])
    
    return fig


def create_multi_panel_animation(results, n_segments=10, pipe_length=10.0):
    """
    Create a multi-panel animation showing temperature profile and time series
    """
    
    times = np.array(results['time'])
    oil_temps_array = np.array(results['oil_temps'])
    positions = np.linspace(0, pipe_length, n_segments)
    
    # Create subplot figure
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Temperature Profile Along Pipe',
            'Outlet Temperature vs Time', 
            'Temperature Gradient',
            'Heat Distribution'
        ),
        specs=[
            [{"secondary_y": False}, {"secondary_y": False}],
            [{"secondary_y": False}, {"secondary_y": False}]
        ]
    )
    
    # Initialize traces for first frame
    initial_temps = oil_temps_array[0]
    outlet_temps = oil_temps_array[:, -1]  # Last segment temperatures
    
    # Main temperature profile
    fig.add_trace(
        go.Scatter(
            x=positions, y=initial_temps,
            mode='lines+markers',
            name='Oil Temperature',
            line=dict(width=3, color='red')
        ),
        row=1, col=1
    )
    
    # Outlet temperature time series
    fig.add_trace(
        go.Scatter(
            x=times[:1], y=outlet_temps[:1],
            mode='lines+markers',
            name='Outlet Temp',
            line=dict(width=2, color='blue')
        ),
        row=1, col=2
    )
    
    # Temperature gradient
    initial_gradient = np.gradient(initial_temps, positions)
    fig.add_trace(
        go.Scatter(
            x=positions, y=initial_gradient,
            mode='lines+markers',
            name='Temperature Gradient',
            line=dict(width=2, color='green')
        ),
        row=2, col=1
    )
    
    # Heat distribution (as bar chart)
    heat_flux = initial_temps * 100  # Simplified heat representation
    fig.add_trace(
        go.Bar(
            x=positions, y=heat_flux,
            name='Heat Content',
            marker_color='orange'
        ),
        row=2, col=2
    )
    
    # Create frames for animation
    frames = []
    for i, t in enumerate(times):
        current_temps = oil_temps_array[i]
        current_gradient = np.gradient(current_temps, positions)
        current_heat = current_temps * 100
        
        frame = go.Frame(
            data=[
                # Temperature profile
                go.Scatter(
                    x=positions, y=current_temps,
                    mode='lines+markers',
                    line=dict(width=3, color='red')
                ),
                # Outlet temperature time series (cumulative)
                go.Scatter(
                    x=times[:i+1], y=outlet_temps[:i+1],
                    mode='lines+markers',
                    line=dict(width=2, color='blue')
                ),
                # Temperature gradient
                go.Scatter(
                    x=positions, y=current_gradient,
                    mode='lines+markers',
                    line=dict(width=2, color='green')
                ),
                # Heat distribution
                go.Bar(
                    x=positions, y=current_heat,
                    marker_color='orange'
                )
            ],
            name=str(i),
            layout=go.Layout(
                title=f"Solar Collector Analysis - Time: {t:.1f}s"
            )
        )
        frames.append(frame)
    
    fig.frames = frames
    
    # Update layout with animation controls
    fig.update_layout(
        title="Comprehensive Solar Collector Temperature Analysis",
        height=600,
        showlegend=True,
        updatemenus=[{
            "type": "buttons",
            "direction": "left", 
            "buttons": [
                {
                    "args": [None, {"frame": {"duration": 300, "redraw": True}}],
                    "label": "Play",
                    "method": "animate"
                },
                {
                    "args": [[None], {"frame": {"duration": 0, "redraw": True}}],
                    "label": "Pause", 
                    "method": "animate"
                }
            ],
            "pad": {"r": 10, "t": 87},
            "showactive": False,
            "x": 0.011, "y": 0
        }],
        sliders=[{
            "active": 0,
            "currentvalue": {"prefix": "Time: ", "suffix": "s"},
            "pad": {"b": 10, "t": 50},
            "len": 0.9, "x": 0.1, "y": 0,
            "steps": [
                {
                    "args": [[str(i)], {"frame": {"duration": 100}}],
                    "label": f"{t:.1f}",
                    "method": "animate"
                }
                for i, t in enumerate(times)
            ]
        }]
    )
    
    # Update axis labels
    fig.update_xaxes(title_text="Position [m]", row=1, col=1)
    fig.update_xaxes(title_text="Time [s]", row=1, col=2)
    fig.update_xaxes(title_text="Position [m]", row=2, col=1)
    fig.update_xaxes(title_text="Position [m]", row=2, col=2)
    
    fig.update_yaxes(title_text="Temperature [°C]", row=1, col=1)
    fig.update_yaxes(title_text="Temperature [°C]", row=1, col=2)
    fig.update_yaxes(title_text="dT/dx [°C/m]", row=2, col=1)
    fig.update_yaxes(title_text="Heat Content [W]", row=2, col=2)
    
    return fig


def create_3d_surface_animation(results, n_segments=10, pipe_length=10.0):
    """
    Create a 3D surface animation showing temperature evolution
    """
    
    times = np.array(results['time'])
    oil_temps_array = np.array(results['oil_temps'])
    positions = np.linspace(0, pipe_length, n_segments)
    
    # Create meshgrid for 3D surface
    T_mesh, X_mesh = np.meshgrid(times, positions)
    Z_mesh = oil_temps_array.T  # Transpose and convert to Celsius
    
    # Create 3D surface plot
    fig = go.Figure()
    
    # Add surface
    fig.add_trace(
        go.Surface(
            x=T_mesh,
            y=X_mesh, 
            z=Z_mesh,
            colorscale='Hot',
            name='Temperature Surface',
            colorbar=dict(title="Temperature [°C]")
        )
    )
    
    # Add animated line trace for current time
    for i, t in enumerate(times):
        fig.add_trace(
            go.Scatter3d(
                x=[t] * len(positions),
                y=positions,
                z=oil_temps_array[i] - 273.15,
                mode='lines+markers',
                line=dict(width=8, color='red'),
                marker=dict(size=6, color='darkred'),
                name=f'Current Profile (t={t:.1f}s)',
                visible=(i == 0)  # Only first trace visible initially
            )
        )
    
    # Create frames for the moving line
    frames = []
    for i, t in enumerate(times):
        frame_data = [
            go.Surface(
                x=T_mesh, y=X_mesh, z=Z_mesh,
                colorscale='Hot',
                showscale=False
            ),
            go.Scatter3d(
                x=[t] * len(positions),
                y=positions,
                z=oil_temps_array[i],
                mode='lines+markers',
                line=dict(width=8, color='red'),
                marker=dict(size=6, color='darkred')
            )
        ]
        frames.append(go.Frame(data=frame_data, name=str(i)))
    
    fig.frames = frames
    
    # Update layout
    fig.update_layout(
        title="3D Temperature Evolution in Solar Collector",
        scene=dict(
            xaxis_title="Time [s]",
            yaxis_title="Position [m]", 
            zaxis_title="Temperature [°C]",
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.2))
        ),
        width=800,
        height=600,
        updatemenus=[{
            "type": "buttons",
            "buttons": [
                {
                    "args": [None, {"frame": {"duration": 300}}],
                    "label": "Play",
                    "method": "animate"
                },
                {
                    "args": [[None], {"frame": {"duration": 0}}],
                    "label": "Pause",
                    "method": "animate"
                }
            ]
        }]
    )
    
    return fig


# Example usage for Jupyter notebook
def display_animations_in_jupyter(results, n_segments=10, pipe_length=10.0):
    """
    Display all animations in a Jupyter notebook
    
    Usage in Jupyter cell:
    ----------------------
    # After running your simulation code:
    display_animations_in_jupyter(results, n_segments, pipe_length)
    """
    
    print("Creating Solar Collector Temperature Animations...")
    
    # 1. Basic temperature profile animation
    print("\n1. Temperature Profile Animation:")
    fig1 = create_solar_collector_animation(results, n_segments, pipe_length)
    fig1.show()
    
    # # 2. Multi-panel analysis
    # print("\n2. Comprehensive Analysis:")
    # fig2 = create_multi_panel_animation(results, n_segments, pipe_length)
    # fig2.show()
    
    # # 3. 3D surface visualization
    # print("\n3. 3D Temperature Surface:")
    # fig3 = create_3d_surface_animation(results, n_segments, pipe_length)
    # fig3.show()
    
    print("\nAnimations created! Use the play buttons and sliders to explore.")


# For direct use in Jupyter notebook:
"""
JUPYTER NOTEBOOK CELL:
----------------------

# Your simulation code here...
t_stop = 60  # seconds
solar_irradiance = 1000  # W/m^2
oil_inlet_temp = 273
oil_temps = cas.DM(np.full(n_segments, 273))  # deg C
amb_temp = 25  # deg C
flow_rate = 0.2  # m^3/s

# Dictionary to store results
results = defaultdict(list)

times = [0.0]
t = 0.0
while True:
    results['time'].append(t)
    results['oil_temps'].append(np.array(oil_temps).flatten())
    dt, oil_temps = calculate_oil_temps_kp1(oil_inlet_temp, oil_temps, 
                                           solar_irradiance, amb_temp, flow_rate)
    t += float(dt)
    if t > t_stop:
        break

# Create and display animations
display_animations_in_jupyter(results, n_segments=10, pipe_length=10.0)
"""

In [ ]:
display_animations_in_jupyter(results, n_segments=n_segments, pipe_length=LENGTH)